In [1]:
import os
import glob
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from NesrHydrusAnalyst import *

In [2]:
src = '../Datasets/H3D2_SandDitch0011'

# Checking past implementations 

In [3]:
# Reading the mesh files

df = read_hydrus_data(folder=src, save_to_csv=False, read_velocities=True)
display(df.sample(3), get_full_simulation_info(df))

 For the entire simulation
Time steps : [0, 5, 15, 30, 60, 120, 180, 240, 500, 720, 1440] 
Dimensions : {'x': (-0.839105, 49.9601), 'y': (0.0, 20.0), 'z': (0.0, 22.9811)} 
Matrix dims: x_vals(41,), z_vals(46,), X(46, 41), Z(46, 41), M(46, 41)


 Variables statistics:


,n,x,y,z,Th_T0,Th_T5,Th_T15,Th_T30,Th_T60,Th_T120,...,V3_T240,V1_T500,V2_T500,V3_T500,V1_T720,V2_T720,V3_T720,V1_T1440,V2_T1440,V3_T1440
5889,5889,33.0295,1.7500,5.78957,0.20,0.200000,0.200000,0.200000,0.225748,0.340256,...,0.000871,0.000161,0.000003,0.000155,0.000073,0.000001,-0.000001,-0.000004,8.290320e-09,-0.000054
7036,7036,30.0295,13.7500,16.28960,0.20,0.200000,0.205818,0.293698,0.380557,0.361741,...,-0.000470,0.000007,0.000002,-0.000201,-0.000003,0.000001,-0.000123,-0.000012,3.278720e-07,-0.000044
3300,3300,26.2829,18.2011,6.83530,0.15,0.062977,0.064463,0.165936,0.430000,0.289751,...,-0.004416,0.000061,-0.000022,-0.001228,-0.000019,-0.000012,-0.000718,-0.000061,-3.171170e-06,-0.000250


,Min,Mean,Median,Max
Th,0.049132,0.257289,3.117305e-01,0.430000
H,-676.912000,-66.645863,-1.962600e+01,12.429200
V1,-1.660930,-0.021988,-1.864930e-05,0.802749
V2,-0.263769,-0.000331,1.837060e-08,0.265775
V3,-1.086420,-0.002844,-2.684000e-05,0.777201


In [4]:
# Rotating the mesh file

df_rotated = rotate_back(df, 2.2899, rotation_axis='y')
get_full_simulation_info(df_rotated)

 For the entire simulation
Time steps : [0, 5, 15, 30, 60, 120, 180, 240, 500, 720, 1440] 
Dimensions : {'x': (0.0, 50.0), 'y': (0.0, 20.0), 'z': (0.0, 21.0)} 
Matrix dims: x_vals(41,), z_vals(43,), X(43, 41), Z(43, 41), M(43, 41)


 Variables statistics:


,Min,Mean,Median,Max
Th,0.049132,0.257289,3.117305e-01,0.430000
H,-676.912000,-66.645863,-1.962600e+01,12.429200
V1,-1.660930,-0.021988,-1.864930e-05,0.802749
V2,-0.263769,-0.000331,1.837060e-08,0.265775
V3,-1.086420,-0.002844,-2.684000e-05,0.777201


In [5]:
#Reading two files: `A_Level.out` and `ATMOSPH.IN`

read_a_level_out(src, geom='3D')

,Time,CumQAP,CumQRP,CumQA,CumQR,CumQ3,hAtm,hRoot,hKode3,A-level,...,rRoot,hCritA,rt,ht,rt,ht,rt,ht,rt,ht
0,60.0,0.0,0.0,0.0,0.0,-2400.0,-4.0,0.0,10.7,1,...,0.000278,10000,-2,0,0,0,0,0,0,0
1,1440.0,0.0,0.0,0.0,0.0,-2400.0,-28.2,0.0,-25.9,2,...,0.000278,10000,0,0,0,0,0,0,0,0


In [6]:
# Reading of one file: Balance.out
    # Returns
    # 1. the total computing time of the simulation AND
    # 2. A dataframe table of water balance info

display (*read_balance_out(src))

'80.7799999713898'

,Time,Volume,VolumeW,InFlow,hMean,WatBalT,WatBalR
0,0.0,21000.0,3798.8,0.000000,-189.990,NaN,NaN
1,5.0,21000.0,3998.8,39.996000,-123.710,0.030824,0.008
2,15.0,21000.0,4398.8,39.998000,-112.810,-0.000182,0.000
3,30.0,21000.0,4998.8,40.001000,-98.652,-0.004614,0.000
4,60.0,21000.0,6198.7,39.999000,-67.802,-0.099587,0.004
5,120.0,21000.0,6191.2,-0.242520,-45.578,-0.225750,0.009
6,180.0,21000.0,6178.2,-0.183360,-37.625,-0.190910,0.008
7,240.0,21000.0,6169.4,-0.127460,-32.564,-0.179100,0.007
8,500.0,21000.0,6151.4,-0.041062,-23.068,-0.148190,0.006
9,720.0,21000.0,6144.9,-0.023799,-20.772,-0.141510,0.006


In [7]:
#Reading four files: SELECTOR.IN, DIMENSIO.IN, Run_Inf.out, Balance.out

read_selector_in(src, geom='3D')

,SimulTime_s,L_Unit,T_Unit,Category,MaxIt,TolTh,TolH,InitH/W,lWat,lChem,...,TLevel_i,Time_i,dt_i,Iter_i,ItCum_i,TLevel_e,Time_e,dt_e,Iter_e,ItCum_e
0,80.78,1.0,1.0,3.0,10.0,0.001,1.0,1.0,1.0,0.0,...,1.0,0.001,0.001,2.0,2.0,565.0,1440.0,114.75,3.0,2214.0


In [8]:
# Combines the most important info from the following files 
# into one dataframe: SELECTOR.IN, DIMENSIO.IN, Run_Inf.out, 
# Balance.out, A_Level.out and ATMOSPH.IN
    
goldf = get_one_line_df(src, simulation_name="Sand Ditch simulation", dims='3d').T
goldf

,SimulTime_s,L_Unit,T_Unit,Category,MaxIt,TolTh,TolH,InitH/W,lWat,lChem,...,hMean5,hMean15,hMean30,hMean60,hMean120,hMean180,hMean240,hMean500,hMean720,hMean1440
Sand Ditch simulation,80.78,1.0,1.0,3.0,10.0,0.001,1.0,1.0,1.0,0.0,...,-123.71,-112.81,-98.652,-67.802,-45.578,-37.625,-32.564,-23.068,-20.772,-19.604


In [9]:
'VarFluxArea' in list(goldf)

False

In [10]:
# updates the get_one_line_df function to add info from 
#     `Boundary.out` and `Check.out` files

golp = get_one_line_plus(src).T
golp

,SimulTime_s,L_Unit,T_Unit,Category,MaxIt,TolTh,TolH,InitH/W,lWat,lChem,...,Alfa_3,n_1,n_2,n_3,Ks_1,Ks_2,Ks_3,Lambda_1,Lambda_2,Lambda_3
Sand Ditch simulation,80.78,1.0,1.0,3.0,10.0,0.001,1.0,1.0,1.0,0.0,...,0.145,2.68,1.48,2.68,0.495,0.00218,2.0,0.5,0.5,0.5


In [11]:
'VarFluxArea' in list(golp)

True

In [12]:
# Reading and combining Cum_Q.out, h_Mean.out, v_Mean.out, Run_inf.out tables

get_mean_outs_table(src)

,Time_T,CumQAP_L3,CumQRP_L3,CumQA_L3,CumQR_L3,CumQ3_L3,CumQ1_L3,CumQS_L3,CumQ5_L3,CumQ6_L3,...,vKode7_L3pT,vKode8_L3pT,vKode9_L3pT,RunOff_LpT,Evapor_LpT,Infiltr_LpT,SnowLayer_L,dt,Iter,ItCum
0,0.0010,0.0,0.0,0.0,0.0,-0.040,0.0,0.0,0.0,0.000009,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001000,2,2
1,0.0020,0.0,0.0,0.0,0.0,-0.080,0.0,0.0,0.0,0.000019,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001000,2,4
2,0.0033,0.0,0.0,0.0,0.0,-0.132,0.0,0.0,0.0,0.000031,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001300,3,7
3,0.0050,0.0,0.0,0.0,0.0,-0.200,0.0,0.0,0.0,0.000046,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001690,3,10
4,0.0072,0.0,0.0,0.0,0.0,-0.287,0.0,0.0,0.0,0.000067,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002197,3,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,980.9964,0.0,0.0,0.0,0.0,-2400.000,0.0,0.0,0.0,58.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76.501000,3,2201
561,1095.7473,0.0,0.0,0.0,0.0,-2400.000,0.0,0.0,0.0,60.100000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114.750000,4,2205
562,1210.4982,0.0,0.0,0.0,0.0,-2400.000,0.0,0.0,0.0,61.600000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114.750000,3,2208
563,1325.2491,0.0,0.0,0.0,0.0,-2400.000,0.0,0.0,0.0,63.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114.750000,3,2211


In [13]:
# Saving the dataframe as CSV

save_this(df, src, 'Original_Grid', authorized=True)
save_this(df_rotated, src, 'Rotated_grid', authorized=True)

**SAVED**
path: ../Datasets/H3D2_SandDitch0011/Nesr/Original_Grid.csv
**SAVED**
path: ../Datasets/H3D2_SandDitch0011/Nesr/Rotated_grid.csv


In [14]:
# Saving all the properties from `get_one_line_plus` to CSV
save_all_simulation_info(
    src, saving_name='Simulation_properties', save_type='csv')

**SAVED**
path: ../Datasets/H3D2_SandDitch0011/Nesr/Simulation_properties.csv


In [15]:
get_materials_properties(src)

,Material,theta,h,log_h,C,K,log_K,S
0,1,0.430,-0.0100,-2.000,0.000000e+00,4.950000e-01,-0.3054,1.0
1,1,0.430,-0.0115,-1.939,0.000000e+00,4.950000e-01,-0.3054,1.0
2,1,0.430,-0.0132,-1.879,0.000000e+00,4.950000e-01,-0.3054,1.0
3,1,0.430,-0.0152,-1.818,0.000000e+00,4.950000e-01,-0.3054,1.0
4,1,0.430,-0.0175,-1.758,0.000000e+00,4.950000e-01,-0.3054,1.0
...,...,...,...,...,...,...,...,...
291,3,0.045,-5720.0000,3.758,1.442000e-09,8.241000e-19,-18.0800,0.0
292,3,0.045,-6580.0000,3.818,9.923000e-10,3.469000e-19,-18.4600,0.0
293,3,0.045,-7560.0000,3.879,6.827000e-10,1.460000e-19,-18.8400,0.0
294,3,0.045,-8700.0000,3.939,4.697000e-10,6.148000e-20,-19.2100,0.0


In [16]:
save_materials_properties(src)

**SAVED**
path: ../Datasets/H3D2_SandDitch0011/Nesr/Materials_info.csv


# Implementing new codes

In [17]:
df

,n,x,y,z,Th_T0,Th_T5,Th_T15,Th_T30,Th_T60,Th_T120,...,V3_T240,V1_T500,V2_T500,V3_T500,V1_T720,V2_T720,V3_T720,V1_T1440,V2_T1440,V3_T1440
1,1,0.00000,0.000000,0.00000,0.15,0.056079,0.056170,0.055658,0.054891,0.382578,...,-0.001520,-0.002441,2.168210e-04,-0.000374,-0.001461,1.225300e-04,-0.000155,-7.349250e-04,7.343800e-05,-0.000111
2,2,49.96010,0.000000,1.99787,0.15,0.053222,0.053808,0.053949,0.053876,0.266495,...,0.001292,0.000226,-1.271570e-04,0.000523,0.000155,-5.971570e-05,0.000259,3.502090e-06,6.466700e-06,-0.000018
3,3,49.88020,0.000000,3.99627,0.15,0.203979,0.208893,0.211409,0.213525,0.359747,...,0.001145,-0.000011,3.776400e-06,0.000386,-0.000004,1.593650e-06,0.000190,5.496710e-07,1.750230e-07,-0.000009
4,4,27.39810,0.000000,3.09723,0.15,0.245272,0.261515,0.270286,0.390000,0.381931,...,-0.000048,0.000024,1.778370e-07,-0.000035,0.000013,-5.810480e-07,-0.000030,-1.227200e-05,-5.683590e-07,-0.000012
5,5,26.67890,0.000000,21.08290,0.10,0.072666,0.394500,0.417685,0.430000,0.202791,...,-0.000808,-0.000424,1.571110e-05,-0.000152,-0.000180,6.295540e-06,-0.000067,-4.969980e-05,1.039020e-06,-0.000015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8246,8246,7.56262,2.354410,21.05230,0.10,0.083682,0.074248,0.069010,0.288235,0.163570,...,-0.000081,-0.000179,1.870280e-06,-0.000026,-0.000102,1.006930e-06,-0.000015,-5.096660e-05,1.499480e-07,-0.000005
8247,8247,5.98613,0.612123,20.52220,0.10,0.079384,0.071694,0.067322,0.261839,0.164561,...,-0.000204,-0.000171,-2.124220e-05,-0.000060,-0.000092,-1.044650e-05,-0.000034,-4.491430e-05,-2.129470e-06,-0.000010
8248,8248,17.91590,0.769231,21.23280,0.10,0.083367,0.074060,0.297446,0.404467,0.183019,...,-0.000093,-0.000291,-1.132860e-05,-0.000033,-0.000184,-5.312390e-06,-0.000019,-7.438720e-05,-8.219470e-07,-0.000006
8249,8249,10.42190,0.769231,20.93320,0.10,0.083720,0.073962,0.071853,0.342952,0.173522,...,-0.000141,-0.000234,-2.296850e-05,-0.000047,-0.000145,-1.106580e-05,-0.000027,-6.965930e-05,-1.706210e-06,-0.000008


In [32]:
melted = df.melt(id_vars=['n', 'x', 'y', 'z'], var_name='VarTime', value_name='Value')
melted

,n,x,y,z,VarTime,Value
0,1,0.00000,0.000000,0.00000,Th_T0,0.150000
1,2,49.96010,0.000000,1.99787,Th_T0,0.150000
2,3,49.88020,0.000000,3.99627,Th_T0,0.150000
3,4,27.39810,0.000000,3.09723,Th_T0,0.150000
4,5,26.67890,0.000000,21.08290,Th_T0,0.100000
...,...,...,...,...,...,...
453745,8246,7.56262,2.354410,21.05230,V3_T1440,-0.000005
453746,8247,5.98613,0.612123,20.52220,V3_T1440,-0.000010
453747,8248,17.91590,0.769231,21.23280,V3_T1440,-0.000006
453748,8249,10.42190,0.769231,20.93320,V3_T1440,-0.000008


In [33]:
new = melted["VarTime"].str.split("_", n = 1, expand = True) 
new
new[1].map(lambda x: x.lstrip('T'))


0            0
1            0
2            0
3            0
4            0
          ... 
453745    1440
453746    1440
453747    1440
453748    1440
453749    1440
Name: 1, Length: 453750, dtype: object

In [34]:
# new = data["Name"].str.split(" ", n = 1, expand = True) 
  
# # making separate first name column from new data frame 
melted["Variable"]= new[0] 
  
# # making separate last name column from new data frame 
melted["Time"]= pd.to_numeric(new[1].map(lambda x: x.lstrip('T')) , downcast='integer')
  
# # Dropping old Name columns 
melted.drop(columns =["VarTime"], inplace = True)

In [35]:
display (melted, melted.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453750 entries, 0 to 453749
Data columns (total 7 columns):
n           453750 non-null int64
x           453750 non-null float64
y           453750 non-null float64
z           453750 non-null float64
Value       453750 non-null float64
Variable    453750 non-null object
Time        453750 non-null int16
dtypes: float64(4), int16(1), int64(1), object(1)
memory usage: 21.6+ MB


,n,x,y,z,Value,Variable,Time
0,1,0.00000,0.000000,0.00000,0.150000,Th,0
1,2,49.96010,0.000000,1.99787,0.150000,Th,0
2,3,49.88020,0.000000,3.99627,0.150000,Th,0
3,4,27.39810,0.000000,3.09723,0.150000,Th,0
4,5,26.67890,0.000000,21.08290,0.100000,Th,0
...,...,...,...,...,...,...,...
453745,8246,7.56262,2.354410,21.05230,-0.000005,V3,1440
453746,8247,5.98613,0.612123,20.52220,-0.000010,V3,1440
453747,8248,17.91590,0.769231,21.23280,-0.000006,V3,1440
453748,8249,10.42190,0.769231,20.93320,-0.000008,V3,1440


None

In [36]:
melted = melted[['n', 'x', 'y', 'z', 'Time', 'Variable', 'Value']]

,n,x,y,z,Time,Variable,Value
0,1,0.00000,0.000000,0.00000,0,Th,0.150000
1,2,49.96010,0.000000,1.99787,0,Th,0.150000
2,3,49.88020,0.000000,3.99627,0,Th,0.150000
3,4,27.39810,0.000000,3.09723,0,Th,0.150000
4,5,26.67890,0.000000,21.08290,0,Th,0.100000
...,...,...,...,...,...,...,...
453745,8246,7.56262,2.354410,21.05230,1440,V3,-0.000005
453746,8247,5.98613,0.612123,20.52220,1440,V3,-0.000010
453747,8248,17.91590,0.769231,21.23280,1440,V3,-0.000006
453748,8249,10.42190,0.769231,20.93320,1440,V3,-0.000008


In [42]:
# melted.pivot(index= ['n', 'x', 'y', 'z', 'Time'], columns='Variable')
melted.pivot( columns='Variable')

n                        x                             ... Value  \
Variable   H   Th  V1  V2      V3   H       Th  V1  V2        V3  ...     H   
0        NaN  1.0 NaN NaN     NaN NaN   0.0000 NaN NaN       NaN  ...   NaN   
1        NaN  2.0 NaN NaN     NaN NaN  49.9601 NaN NaN       NaN  ...   NaN   
2        NaN  3.0 NaN NaN     NaN NaN  49.8802 NaN NaN       NaN  ...   NaN   
3        NaN  4.0 NaN NaN     NaN NaN  27.3981 NaN NaN       NaN  ...   NaN   
4        NaN  5.0 NaN NaN     NaN NaN  26.6789 NaN NaN       NaN  ...   NaN   
...       ..  ...  ..  ..     ...  ..      ...  ..  ..       ...  ...   ...   
453745   NaN  NaN NaN NaN  8246.0 NaN      NaN NaN NaN   7.56262  ...   NaN   
453746   NaN  NaN NaN NaN  8247.0 NaN      NaN NaN NaN   5.98613  ...   NaN   
453747   NaN  NaN NaN NaN  8248.0 NaN      NaN NaN NaN  17.91590  ...   NaN   
453748   NaN  NaN NaN NaN  8249.0 NaN      NaN NaN NaN  10.42190  ...   NaN   
453749   NaN  NaN NaN NaN  8250.0 NaN      NaN NaN NaN   2.92788  ...   NaN   

                                 Time                       
Variable    Th  V1  V2        V3    H   Th  V1  V2      V3  
0         0.15 NaN NaN       NaN  NaN  0.0 NaN NaN     NaN  
1         0.15 NaN NaN       NaN  NaN  0.0 NaN NaN     NaN  
2         0.15 NaN NaN       NaN  NaN  0.0 NaN NaN     NaN  
3         0.15 NaN NaN       NaN  NaN  0.0 NaN NaN     NaN  
4         0.10 NaN NaN       NaN  NaN  0.0 NaN NaN     NaN  
...        ...  ..  ..       ...  ...  ...  ..  ..     ...  
453745     NaN NaN NaN -0.000005  NaN  NaN NaN NaN  1440.0  
453746     NaN NaN NaN -0.000010  NaN  NaN NaN NaN  1440.0  
453747     NaN NaN NaN -0.000006  NaN  NaN NaN NaN  1440.0  
453748     NaN NaN NaN -0.000008  NaN  NaN NaN NaN  1440.0  
453749     NaN NaN NaN -0.000006  NaN  NaN NaN NaN  1440.0  

[453750 rows x 30 columns]

In [45]:
melted.groupby(['n', 'x', 'y', 'z', 'Time', 'Variable'])['Value'].sum().unstack(level=-1)

Variable                                  H        Th        V1            V2  \
n    x       y        z       Time                                              
1    0.00000 0.000000 0.0000  0    -14.4309  0.150000 -0.026989  0.000000e+00   
                              5    -57.6896  0.056079 -0.000012 -8.646400e-06   
                              15   -57.4075  0.056170 -0.000011 -5.722230e-06   
                              30   -59.0396  0.055658 -0.000008 -3.396810e-06   
                              60   -61.7308  0.054891 -0.000006 -1.689270e-06   
...                                     ...       ...       ...           ...   
8250 2.92788 0.769231 20.6335 180  -17.0388  0.126655 -0.001140 -8.060170e-05   
                              240  -19.1639  0.112985 -0.000586 -4.799530e-05   
                              500  -25.4477  0.088089 -0.000116 -1.143710e-05   
                              720  -28.9992  0.079795 -0.000056 -5.135350e-06   
                              1440 -35.1176  0.070359 -0.000024 -8.426380e-07   

Variable                                  V3  
n    x       y        z       Time            
1    0.00000 0.000000 0.0000  0     0.672859  
                              5     0.000025  
                              15    0.000021  
                              30    0.000015  
                              60    0.000010  
...                                      ...  
8250 2.92788 0.769231 20.6335 180  -0.000237  
                              240  -0.000141  
                              500  -0.000039  
                              720  -0.000022  
                              1440 -0.000006  

[90750 rows x 5 columns]

In [47]:
melted.groupby(['n', 'x', 'y', 'z', 'Time', 'Variable'])['Value'].sum().unstack(level=-2)

Time                                            0          5          15    \
n    x       y        z       Variable                                       
1    0.00000 0.000000 0.0000  H        -1.443090e+01 -57.689600 -57.407500   
                              Th        1.500000e-01   0.056079   0.056170   
                              V1       -2.698870e-02  -0.000012  -0.000011   
                              V2        0.000000e+00  -0.000009  -0.000006   
                              V3        6.728590e-01   0.000025   0.000021   
...                                              ...        ...        ...   
8250 2.92788 0.769231 20.6335 H        -2.189090e+01 -27.157900 -32.263600   
                              Th        1.000000e-01   0.083751   0.074181   
                              V1       -3.098670e-10   0.000076   0.000035   
                              V2        2.023620e-10  -0.000457  -0.000159   
                              V3       -7.345090e-04  -0.001263  -0.000379   

Time                                         30         60         120   \
n    x       y        z       Variable                                    
1    0.00000 0.000000 0.0000  H        -59.039600 -61.730800  -4.280250   
                              Th         0.055658   0.054891   0.382578   
                              V1        -0.000008  -0.000006  -0.013722   
                              V2        -0.000003  -0.000002   0.001488   
                              V3         0.000015   0.000010  -0.000174   
...                                           ...        ...        ...   
8250 2.92788 0.769231 20.6335 H        -36.385500 -16.459700 -14.307100   
                              Th         0.068909   0.131157   0.151332   
                              V1         0.000016  -0.022504  -0.003168   
                              V2        -0.000069  -0.001585  -0.000166   
                              V3        -0.000157  -0.002956  -0.000533   

Time                                         180        240        500   \
n    x       y        z       Variable                                    
1    0.00000 0.000000 0.0000  H         -4.776540  -5.514070  -7.664290   
                              Th         0.367795   0.344225   0.276955   
                              V1        -0.009938  -0.007020  -0.002441   
                              V2         0.001017   0.000709   0.000217   
                              V3        -0.001940  -0.001520  -0.000374   
...                                           ...        ...        ...   
8250 2.92788 0.769231 20.6335 H        -17.038800 -19.163900 -25.447700   
                              Th         0.126655   0.112985   0.088089   
                              V1        -0.001140  -0.000586  -0.000116   
                              V2        -0.000081  -0.000048  -0.000011   
                              V3        -0.000237  -0.000141  -0.000039   

Time                                         720           1440  
n    x       y        z       Variable                           
1    0.00000 0.000000 0.0000  H         -8.797890 -1.001450e+01  
                              Th         0.246083  2.179460e-01  
                              V1        -0.001461 -7.349250e-04  
                              V2         0.000123  7.343800e-05  
                              V3        -0.000155 -1.113610e-04  
...                                           ...           ...  
8250 2.92788 0.769231 20.6335 H        -28.999200 -3.511760e+01  
                              Th         0.079795  7.035860e-02  
                              V1        -0.000056 -2.422880e-05  
                              V2        -0.000005 -8.426380e-07  
                              V3        -0.000022 -6.312230e-06  

[41250 rows x 11 columns]

In [48]:
melted.groupby(['n', 'x', 'y', 'z', 'Time', 'Variable'])['Value'].sum().unstack(level=-1).reset_index()

Variable,n,x,y,z,Time,H,Th,V1,V2,V3
0,1,0.00000,0.000000,0.0000,0,-14.4309,0.150000,-0.026989,0.000000e+00,0.672859
1,1,0.00000,0.000000,0.0000,5,-57.6896,0.056079,-0.000012,-8.646400e-06,0.000025
2,1,0.00000,0.000000,0.0000,15,-57.4075,0.056170,-0.000011,-5.722230e-06,0.000021
3,1,0.00000,0.000000,0.0000,30,-59.0396,0.055658,-0.000008,-3.396810e-06,0.000015
4,1,0.00000,0.000000,0.0000,60,-61.7308,0.054891,-0.000006,-1.689270e-06,0.000010
...,...,...,...,...,...,...,...,...,...,...
90745,8250,2.92788,0.769231,20.6335,180,-17.0388,0.126655,-0.001140,-8.060170e-05,-0.000237
90746,8250,2.92788,0.769231,20.6335,240,-19.1639,0.112985,-0.000586,-4.799530e-05,-0.000141
90747,8250,2.92788,0.769231,20.6335,500,-25.4477,0.088089,-0.000116,-1.143710e-05,-0.000039
90748,8250,2.92788,0.769231,20.6335,720,-28.9992,0.079795,-0.000056,-5.135350e-06,-0.000022


In [51]:
def reform_grid(data_frame):
    '''
    Convert the read data_frame to a reformed table with columns:
    [n	x	y	z	Time	H	Th	V1	V2	V3]
    '''
    melted = data_frame.melt(
        id_vars=['n', 'x', 'y', 'z'], var_name='VarTime', value_name='Value')
    new = melted["VarTime"].str.split("_", n=1, expand=True)
    # # making separate first name column from new data frame
    melted["Variable"] = new[0]
    # # making separate last name column from new data frame
    melted["Time"] = pd.to_numeric(
        new[1].map(lambda x: x.lstrip('T')), downcast='integer')
    # # Dropping old Name columns
    melted.drop(columns=["VarTime"], inplace=True)
    # unmelting to make variables as columns, aand time in rows
    return melted.groupby(
        ['n', 'x', 'y', 'z', 'Time',
         'Variable'])['Value'].sum().unstack(level=-1).reset_index()

In [52]:
reform_grid(df_rotated)

Variable,n,x,y,z,Time,H,Th,V1,V2,V3
0,1,0.00,0.000,0.0,0,-14.4309,0.150000,-0.026989,0.000000e+00,0.672859
1,1,0.00,0.000,0.0,5,-57.6896,0.056079,-0.000012,-8.646400e-06,0.000025
2,1,0.00,0.000,0.0,15,-57.4075,0.056170,-0.000011,-5.722230e-06,0.000021
3,1,0.00,0.000,0.0,30,-59.0396,0.055658,-0.000008,-3.396810e-06,0.000015
4,1,0.00,0.000,0.0,60,-61.7308,0.054891,-0.000006,-1.689270e-06,0.000010
...,...,...,...,...,...,...,...,...,...,...
90745,8250,3.75,0.769,20.5,180,-17.0388,0.126655,-0.001140,-8.060170e-05,-0.000237
90746,8250,3.75,0.769,20.5,240,-19.1639,0.112985,-0.000586,-4.799530e-05,-0.000141
90747,8250,3.75,0.769,20.5,500,-25.4477,0.088089,-0.000116,-1.143710e-05,-0.000039
90748,8250,3.75,0.769,20.5,720,-28.9992,0.079795,-0.000056,-5.135350e-06,-0.000022


In [21]:
# # Reading the files `MESHTRIA.TXT`, `TH.TXT`, `V.TXT`, `H.TXT`
# df = read_hydrus_data(folder=src, save_to_csv=False, read_velocities=True)
# # If required, rotate the given dataframe
# df_rotated = rotate_back(df, 2.2899, rotation_axis='y')

# save_this(df, src, 'Original_Grid', authorized=True)
# save_this(df_rotated, src, 'Rotated_grid', authorized=True)

# # Combining the two files: `A_Level.out` and `ATMOSPH.IN`
# save_this(read_a_level_out(src, geom='3D'), src, 'Atm_A_level', authorized=True)

# # Saving info from one file: Balance.out
# save_this(read_balance_out(src)[1], src, 'MassBalance', authorized=True)

# # Saving eight files: SELECTOR.IN, DIMENSIO.IN, Run_Inf.out, Balance.out, 
# # A_Level.out, ATMOSPH.IN, `Boundary.out` and `Check.out` files
# save_all_simulation_info(src,
#                              saving_name='Simulation_info',
#                              save_type='csv')
# # Reading and combining Cum_Q.out, h_Mean.out, v_Mean.out, Run_inf.out tables
# save_this(get_mean_outs_table(src), src, 'BoundaryData', authorized=True)
# # Reading the file `Check.out` for materials properties
# save_this(get_materials_properties(src), src, 'MaterialsData', authorized=True)
